In [88]:
import matplotlib.pyplot as plt
import matplotlib as pmimg
import numpy as np
import skimage
from skimage import color
from skimage import data
from skimage import io
from tqdm import tqdm
import cv2

import os 
dir_path = os.getcwd()

In [89]:
def oneHot(imgTitle):
    animalLabel = imgTitle.split('.')[0][-3:]
    return animalLabel

In [90]:
def createTrainSet():
    trainingSet = []
#     data_dir = '/Users/Jack/Documents/ml/ml_book/DogOrCat/data/practice-train'
    data_dir = '/Users/Jack/Documents/ml/ml_book/DogOrCat/data/alternate_data/train'
    for imgLabel in tqdm(os.listdir(data_dir)[:200]):
        animalLabel = oneHot(imgLabel)
        #dog = [1, 0], cat = [0,1]
        oneHotLabel = []
        if animalLabel == 'dog':
            oneHotLabel = [1, 0]
        elif animalLabel == 'cat':
            oneHotLabel = [0, 1]
        else:
            print('nada')
            continue
        imgPath = os.path.join(data_dir, imgLabel)
        imgObj = cv2.imread(imgPath, cv2.IMREAD_GRAYSCALE)
        imgObj = cv2.resize(imgObj, (10, 10))

        trainingSet.append([imgObj, oneHotLabel])
    return trainingSet

train = createTrainSet()

100%|██████████| 200/200 [00:00<00:00, 775.16it/s]


In [91]:
# Filter
# 
side_detector_layer = np.array([
    [-1, 0, 1],
    [-1, 0, 1],
    [-1, 0, 1]
])

reverse_side_detector_layer = np.array([
    [1, 0, -1],
    [1, 0, -1],
    [1, 0, -1]
])

top_bottom_detector_layer = np.array([
    [1, 1, 1],
    [0, 0, 0],
    [-1, -1, 1]
])

angle_detector_layer = np.array([
    [0, 1, 1],
    [-1, 0, 1],
    [-1, -1, 0]
])


top_detector_layer = np.array([
    [1, 1, 1],
    [-1, -1, -1],
    [-1, -1, -1]
])


x_detector_layer = np.array([
    [1, 0, 1],
    [0, 1, 0],
    [1, 0, 1]
])

right_knight_detector_layer = np.array([
    [1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1],
    [0, 0, 0, 1, 1],
    [0, 0, 0, 1, 1],
    [0, 0, 0, 1, 1]
]);

left_knight_detector_layer = np.array([
    [1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1],
    [1, 1, 0, 0, 0],
    [1, 1, 0, 0, 0],
    [1, 1, 0, 0, 0]
]);

small_angle_detector_layer = np.array([
    [1, 1, 0],
    [1, 0, 1],
    [0, 1, 1],
])

horiz_filter = np.array([
    [1, 1, 1],
    [0, 0, 0],
    [-1, -1, -1],
])
vertical_filter = np.array([
    [1, 0, -1],
    [1, 0, -1],
    [1, 0, -1]
])
whole_filter = np.array([
    [1, 1, 1],
    [1, 1, 1],
    [1, 1, 1]
])

In [92]:
# def max_pool(img, filter, padding, stride, depth):
#     outputDim = len(img[0]) - len(filter[0]) + (2*padding) * stride + 1
#     print('OUTPUT DIM IS:' + str(outputDim))
#     resultArr = np.zeros(((depth), (outputDim), (outputDim)), dtype=int)
#     filterLength = len(filter)
#     for zIdx in range(depth):
#         for yIdx in range(1, outputDim-1):
#             for xIdx in range(1, outputDim-1):
#                 window = np.zeros((filterLength, filterLength), dtype=int)

#                 windowYIdx = 0 + yIdx
#                 while windowYIdx < (len(filter) + yIdx):
#                     windowXEnd = xIdx + len(filter)
#                     currRow = np.array(img[windowYIdx][xIdx:windowXEnd])
#                     window[windowYIdx - yIdx] = currRow
#                     windowYIdx += 1

#                 filteredWindow = window * filter
#                 nodeValue = np.amax(filteredWindow)
#                 resultArr[zIdx][yIdx][xIdx] = nodeValue
    
#     return resultArr

def max_pool(img, filter, padding='same', stride=1, depth=1):
    if padding == 'same':
        padding = int(len(filter)/2)
        
    outputDim = len(img[0]) - len(filter[0]) + (2*padding) * stride + 1
    padded_img = np.pad(img, ((padding, padding)), 'constant', constant_values=(0, 0))
    resultArr = np.zeros((depth, outputDim, outputDim), dtype=int)
    
    filterLength = len(filter)

    for zIdx in range(depth):
        for yIdx in range(padded_img.shape[0] - (2 * padding)):
            for xIdx in range(padded_img.shape[1] - (2 * padding)):
                yEnd = yIdx + len(filter)
                xEnd = xIdx + len(filter)
                windowRows = padded_img[yIdx:yEnd]
                window = np.zeros((len(filter), len(filter)))
                xWin = 0
                for row in windowRows:
                    window[xWin] = row[xIdx:xEnd]
                    xWin += 1
                    
                filteredWindow = window * filter
                nodeValue = np.amax(filteredWindow)
                resultArr[zIdx][yIdx][xIdx] = nodeValue
    #Boom!

In [93]:
def relu(imgMatrix):
    filterMask = imgMatrix > 0
    reluMatrix = imgMatrix * filterMask
    return reluMatrix

In [145]:
def convolve_2d(img, filter, padding='same', stride=1, depth=1):
    if padding == 'same':
        padding = int(len(filter)/2)
        
    outputDim = len(img[0]) - len(filter[0]) + (2*padding) * stride + 1
    padded_img = np.pad(img, ((padding, padding)), 'constant', constant_values=(0, 0))
    resultArr = np.zeros((depth, outputDim, outputDim), dtype=int)
    
    filterLength = len(filter)

    for zIdx in range(depth):
        for yIdx in range(padded_img.shape[0] - (2 * padding)):
            for xIdx in range(padded_img.shape[1] - (2 * padding)):
                yEnd = yIdx + len(filter)
                xEnd = xIdx + len(filter)
                windowRows = padded_img[yIdx:yEnd]
                window = np.zeros((len(filter), len(filter)))
                xWin = 0
                for row in windowRows:
                    window[xWin] = row[xIdx:xEnd]
                    xWin += 1
                    
                filteredWindow = window * filter
                nodeValue = np.average(filteredWindow)
                resultArr[zIdx][yIdx][xIdx] = nodeValue
    #Boom!
    return resultArr

    
def createConvolutionalLayer(filtersArr, img):
    convFilterArr = np.zeros((len(filtersArr), img.shape[0], img.shape[1]))
    idx = 0
    for filter in filtersArr:
        convResult = convolve_2d(img, filter, depth=1)
        convFilterArr[idx] = convResult
        idx += 1
    return convFilterArr    

def train_conv_layer(imgSet, filters):
    imgSet = np.array(imgSet)
    x = 0
    for img in tqdm(imgSet):
        imgSet[x][0] = createConvolutionalLayer(filters, img[0])
        x += 1
    return imgSet

filters = [
    side_detector_layer,
    reverse_side_detector_layer,
    small_angle_detector_layer,
    top_bottom_detector_layer,
    angle_detector_layer,
    top_detector_layer,
    left_knight_detector_layer,
    right_knight_detector_layer,
    x_detector_layer
]


def relu_layer(imgArr):
    imgObjIdx = 0
    for imgObj in imgArr:
        filterIdx = 0
        listOfFilteredImages = imgObj[0]
        for imgMatrix in listOfFilteredImages:
            reluImg = relu(imgMatrix)
            imgObj[0][filterIdx] = reluImg
            filterIdx += 1
        
        imgArr[imgObjIdx] = imgObj
        imgObjIdx += 1  
    return imgArr

animal = 150
imageOne = train[animal][0]
trainSet = train[0:11]
trainSet = train_conv_layer(trainSet, filters)
trainSet = relu_layer(trainSet)

100%|██████████| 11/11 [00:00<00:00, 38.65it/s]


[[array([[[15., 14., 18.,  0.,  1.,  0., -0., -0., 15., -0.],
        [28., 27., 21., -0.,  4., -0., -0., -0., 26., -0.],
        [27., 22., 20., -0., -0., -0., -0., -0., 27., -0.],
        [29., 38., 21., -0., -0., -0., -0., -0., 14., -0.],
        [34., 25., 15., -0., -0., -0.,  7., -0.,  9., -0.],
        [41., 12.,  0., -0., -0., -0., 16.,  6., -0., -0.],
        [39.,  3., -0., -0., -0.,  0., 13., 10., -0., -0.],
        [29., 14.,  3., -0.,  5.,  0., 10., -0., -0., -0.],
        [24., 26., 12., -0., -0., -0., -0., -0., -0., -0.],
        [17., 18.,  9., -0., -0.,  0.,  0., -0., -0., -0.]],

       [[-0., -0., -0.,  0., -0.,  0., 15., 12., -0., 14.],
        [-0., -0., -0.,  6., -0.,  6., 30., 10., -0., 22.],
        [-0., -0., -0.,  5.,  1.,  3., 22.,  8., -0., 22.],
        [-0., -0., -0., 12.,  4.,  7., 12., 13., -0., 17.],
        [-0., -0., -0., 12., 13.,  1., -0., 10., -0., 27.],
        [-0., -0.,  0.,  7.,  3.,  5., -0., -0.,  7., 40.],
        [-0., -0.,  4.,  6.,  1.,  0

In [ ]:
def max_pool_layer(imgArr):
    depth = len(imgArr)
    maxi = max_pool(imgArr[0], vertical_filter, padding=1, stride=1, depth=depth)
    idx = 0
    for img in imgArr:
        print(img)
        imgArr[idx] = max_pool(img, vertical_filter)
    return imgArr

# animal = 4773
# imageOne = train[animal][0]

# convLayer = createConvolutionalLayer(filters, imageOne)
# print(convLayer[0].shape)
    
# relu_matrix = relu_layer(convLayer)
# print(relu_matrix.shape)